# 0.0 Imports

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.display import HTML

## 0.1 Helper Functions

In [6]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline

    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()
    
jupyter_settings()

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


## 0.2 Loading Data

In [7]:
# Loading data

df_raw = pd.read_csv('../data/train.csv')

In [8]:
df_raw.head()

,id,gender,age,region_code,sales_channel,driving_license,vehicle_age,vehicle_damage,vehicle_insured,annual_premium,days_client_associate,interested
0,274711,Male,22,40,152,1,< 1 Year,Yes,0,25455.0,203,0
1,216540,Female,26,8,26,1,< 1 Year,No,1,38393.0,45,0
2,78984,Female,32,28,122,1,< 1 Year,Yes,0,32118.0,35,1
3,218369,Male,32,37,152,1,< 1 Year,Yes,0,37711.0,215,0
4,307216,Female,24,28,156,1,< 1 Year,Yes,0,48706.0,222,0


# 1.0 Data Description